Scenario: Smart Factory Temperature Monitoring
A factory monitors the daily average temperature (°C) of a critical machine.
The goal is to predict tomorrow’s temperature using the last 5 days of temperature readings in order to:
Detect overheating trends
Schedule preventive maintenance
Avoid machine failure
🔹 Sample Dataset (Time-Series)
 
Day	Temperature (°C)
Day 1	68
Day 2	70
Day 3	71
Day 4	73
Day 5	74
Day 6	76
Day 7	78
Day 8	80
Day 9	82
Day 10	85

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequencial
from tensorflow.layers import Dense


In [ ]:


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

tf.random.set_seed(1)
np.random.seed(1)


temps = np.array([68, 70, 71, 73, 74, 76, 78, 80, 82, 85], dtype=np.float32)


window_size = 5
X_list, y_list = [], []
for i in range(len(temps) - window_size):
    X_list.append(temps[i:i+window_size])     # 5 consecutive days
    y_list.append(temps[i+window_size])       # next day's temp

X = np.array(X_list, dtype=np.float32)        # shape: (num_samples, 5)
y = np.array(y_list,  dtype=np.float32)       # shape: (num_samples,)

# Reshape for RNN: (batch, time_steps, features)
X = X[..., np.newaxis]   # (num_samples, 5, 1)
y = y[..., np.newaxis]   # (num_samples, 1)

print("Number of samples:", X.shape[0])  # Expect 5 samples with 10 days and window=5


train_size = max(1, int(0.8 * len(X)))  # 4
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]


x_mean, x_std = X_train.mean(), X_train.std() + 1e-8
y_mean, y_std = y_train.mean(), y_train.std() + 1e-8

X_train_n = (X_train - x_mean) / x_std
X_val_n   = (X_val   - x_mean) / x_std
y_train_n = (y_train - y_mean) / y_std
y_val_n   = (y_val   - y_mean) / y_std


def build_model(units=16):
    model = Sequential([
        SimpleRNN(units, activation='tanh', input_shape=(window_size, 1)),
        Dense(1)  # regression output (normalized scale)
    ])
    return model


sgd_opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, clipnorm=1.0)

model_sgd = build_model(16)
model_sgd.compile(optimizer=sgd_opt, loss='mse', metrics=['mae'])

history_sgd = model_sgd.fit(
    X_train_n, y_train_n,
    epochs=500,          # more epochs to compensate for tiny data
    batch_size=2,
    validation_data=(X_val_n, y_val_n),
    verbose=0
)

print("Final train loss (SGD):", history_sgd.history['loss'][-1])
print("Final val   loss (SGD):", history_sgd.history['val_loss'][-1])


adam_opt = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)

model_adam = build_model(16)
model_adam.compile(optimizer=adam_opt, loss='mse', metrics=['mae'])

history_adam = model_adam.fit(
    X_train_n, y_train_n,
    epochs=500,
    batch_size=2,
    validation_data=(X_val_n, y_val_n),
    verbose=0
)

print("Final train loss (Adam):", history_adam.history['loss'][-1])
print("Final val   loss (Adam):", history_adam.history['val_loss'][-1])


last_5_days = temps[-window_size:]  # [74, 76, 78, 80, 82]
inp = last_5_days.reshape(1, window_size, 1)


inp_n = (inp - x_mean) / x_std

pred_n_sgd  = model_sgd.predict(inp_n,  verbose=0)[0, 0]
pred_n_adam = model_adam.predict(inp_n, verbose=0)[0, 0]


pred_sgd  = pred_n_sgd  * y_std + y_mean
pred_adam = pred_n_adam * y_std + y_mean

print("\nLast 5 days (°C):", np.round(last_5_days, 2))
print(f"Predicted next temp (SGD) : {pred_sgd:.2f} °C")
print(f"Predicted next temp (Adam): {pred_adam:.2f} °C")



Number of samples: 5


c:\Users\Pratik.Jadhav\Downloads\Training\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Final train loss (SGD): 2.220446049250313e-16
Final val   loss (SGD): 1.1457267999649048
Final train loss (Adam): 1.354472090042691e-14
Final val   loss (Adam): 1.0527070760726929

Last 5 days (°C): [76. 78. 80. 82. 85.]
Predicted next temp (SGD) : 82.78 °C
Predicted next temp (Adam): 83.19 °C


In [ ]:


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

tf.random.set_seed(1)
np.random.seed(1)


temps = np.array([30,31,30,32,33,29,30,32,32,32], dtype=np.float32)


window_size = 7
X_list, y_list = [], []
for i in range(len(temps) - window_size):
    X_list.append(temps[i:i+window_size])     # 5 consecutive days
    y_list.append(temps[i+window_size])       # next day's temp

X = np.array(X_list, dtype=np.float32)        # shape: (num_samples, 5)
y = np.array(y_list,  dtype=np.float32)       # shape: (num_samples,)

# Reshape for RNN: (batch, time_steps, features)
X = X[..., np.newaxis]   # (num_samples, 5, 1)
y = y[..., np.newaxis]   # (num_samples, 1)

print("Number of samples:", X.shape[0])  # Expect 5 samples with 10 days and window=5


train_size = max(1, int(0.8 * len(X)))  # 4
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

x_mean, x_std = X_train.mean(), X_train.std() + 1e-8
y_mean, y_std = y_train.mean(), y_train.std() + 1e-8

X_train_n = (X_train - x_mean) / x_std
X_val_n   = (X_val   - x_mean) / x_std
y_train_n = (y_train - y_mean) / y_std
y_val_n   = (y_val   - y_mean) / y_std

def build_model(units=16):
    model = Sequential([
        SimpleRNN(units, activation='tanh', input_shape=(window_size, 1)),
        Dense(1)  # regression output (normalized scale)
    ])
    return model

sgd_opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, clipnorm=1.0)

model_sgd = build_model(16)
model_sgd.compile(optimizer=sgd_opt, loss='mse', metrics=['mae'])

history_sgd = model_sgd.fit(
    X_train_n, y_train_n,
    epochs=500,          # more epochs to compensate for tiny data
    batch_size=2,
    validation_data=(X_val_n, y_val_n),
    verbose=0
)

print("Final train loss (SGD):", history_sgd.history['loss'][-1])
print("Final val   loss (SGD):", history_sgd.history['val_loss'][-1])

adam_opt = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)

model_adam = build_model(16)
model_adam.compile(optimizer=adam_opt, loss='mse', metrics=['mae'])

history_adam = model_adam.fit(
    X_train_n, y_train_n,
    epochs=500,
    batch_size=2,
    validation_data=(X_val_n, y_val_n),
    verbose=0
)

print("Final train loss (Adam):", history_adam.history['loss'][-1])
print("Final val   loss (Adam):", history_adam.history['val_loss'][-1])


last_5_days = temps[-window_size:]  # [74, 76, 78, 80, 82]
inp = last_5_days.reshape(1, window_size, 1)


inp_n = (inp - x_mean) / x_std

pred_n_sgd  = model_sgd.predict(inp_n,  verbose=0)[0, 0]
pred_n_adam = model_adam.predict(inp_n, verbose=0)[0, 0]


pred_sgd  = pred_n_sgd  * y_std + y_mean
pred_adam = pred_n_adam * y_std + y_mean

print("\nLast 5 days (°C):", np.round(last_5_days, 2))
print(f"Predicted next temp (SGD) : {pred_sgd:.2f} °C")
print(f"Predicted next temp (Adam): {pred_adam:.2f} °C")



Number of samples: 3
Final train loss (SGD): 2.498001805406602e-16
Final val   loss (SGD): 0.0011678278679028153
Final train loss (Adam): 1.6709723882346594e-14
Final val   loss (Adam): 0.09155550599098206

Last 5 days (°C): [32. 33. 29. 30. 32. 32. 32.]
Predicted next temp (SGD) : 32.00 °C
Predicted next temp (Adam): 32.00 °C
